# Google Colab Connect

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import statements 

In [1]:
from sklearn.cluster import KMeans
import numpy as np
import cv2
import math
import scipy.spatial.distance as d



In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure 
import json
import datetime
import copy
from PIL import Image as im 
import joblib


In [3]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [4]:
import scipy.cluster.vq as vq
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

# Saving and Loading function 

In [5]:
# Saving and Loading models using joblib 
def save(filename, obj):
  with open(filename, 'wb') as handle:
      joblib.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load(filename):
  with open(filename, 'rb') as handle:
      return joblib.load(filename)

# Data Loaders and data class 

In [6]:
# Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomSizedCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:886: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  warnings.warn("The use of the transforms.RandomSizedCrop transform is deprecated, " +


In [7]:
# custom class to provide the image name along with the image 
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image name
        img_name = self.imgs[index][0].split("/")[-1]
        
        # print(path)
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (img_name,))
        return tuple_with_path

In [ ]:
# instantiate the dataset and dataloader
# Train 
# root directory of train , test and val needs to be put here  
train_data_dir = '/content/drive/MyDrive/SUB/PROJECTS/IR/myData'
train_dataset = ImageFolderWithPaths(train_data_dir, transform=train_transforms) 

In [ ]:
batch = 64
train_dataloader = torch.utils.data.DataLoader(train_dataset ,batch_size=batch )

In [ ]:
len(next(iter(train_dataloader)))

3

# CNN feature extractor class 

In [8]:
# Class to extract features from images 
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        VGG = models.vgg16(pretrained=True)
        self.feature = VGG.features
        self.classifier = nn.Sequential(*list(VGG.classifier.children())[:-2])
        pretrained_dict = VGG.state_dict()
        model_dict = self.classifier.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict)
        self.classifier.load_state_dict(model_dict)

        # self.pool = nn.MaxPool2d(2, stride=3) 
        self.cnn1 = nn.Conv2d(512, 250, 1)
        
    def forward(self, x):
        output = self.feature(x)
        output = self.cnn1(output)
        return output


In [9]:
extractor = Encoder()
print(extractor)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth



Encoder(
  (feature): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilati

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Feature Extraction and saving 

In [ ]:
# temp = []
# image_names = []
# extractor.to(device)

# count = 0
# for idx,loader in enumerate([train_dataloader]):
#   # labels got here are of no use 
#   print("--------------------doing------------")
#   for inputs, labels, paths in loader:
#     # print(inputs, labels, paths)
#     inputs = inputs.to(device)
#     feature_outputs = extractor(inputs)
  
#     for f, p in zip(feature_outputs, paths):
#       count = count + 1
#       print(count)
#       # specify your own path to save 
#       np.save( '/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/img_feaures/' + p , f.cpu().detach().numpy())
#       temp.append(f.cpu().detach().numpy())
#       image_names.append(p)


In [ ]:
# All images features appended in temp and each of following shape 
# print(temp[0].shape) , print(len(image_names))

(250, 7, 7)
5063


(None, None)

In [ ]:
# save("/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/others/all_img_features.pkl",temp)
# save("/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/others/all_img_names.pkl",image_names)

# Diving Images into train and test 

In [ ]:
# train_path = "/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/train/"
# test_path = "/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/test/"
# original_path = "/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/img_feaures/"

In [ ]:
# import shutil 
# # moving in train 
# train_f = []
# train_name = []

# imgs = os.listdir(original_path)
# size = len(imgs)
# print(size)

# np.random.seed(0)
# np.random.shuffle(imgs)

# other = imgs[:int(1*size)]
# other = imgs[:4950]
# print(len(other))

# for img in other:
#   train_name.append(".".join(img.split('.')[:-1]))
#   train_f.append(np.load(original_path + img))
#   shutil.move(original_path + img, train_path) 

# # train images temp diff and names of images different 

In [ ]:
# import shutil 
# # moving in test
# test_f = []
# test_name = []

# imgs = os.listdir(original_path)
# size = len(imgs)
# print(size)

# np.random.seed(0)
# np.random.shuffle(imgs)

# other = imgs[:int(1*size)]
# print(len(other))

# for img in other:
#   test_name.append(".".join(img.split('.')[:-1]))
#   test_f.append(np.load(original_path + img ))
#   shutil.move(original_path + img, test_path) 

  

113
113


In [ ]:
# save("/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/others/train_name.pkl",train_name)
# save("/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/others/train_f.pkl",train_f)
# save("/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/others/test_name.pkl",test_name)
# save("/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/others/test_f.pkl",test_f)

# Generating k-mean data 

In [15]:
train_f = load("/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/others/train_f.pkl") 
train_name = load("/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/others/train_name.pkl")
test_f = load("/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/others/test_f.pkl") 
test_name = load("/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/others/test_name.pkl")

# # With temp we create data points for kmeans 
# for idx,t in enumerate(train_f):
#   t_new = np.reshape(t,(250,7*7)).T
#   if idx == 0:
#     t_old = np.reshape(t,(250,7*7)).T
#   else:
#     t_old = np.append(t_old, t_new, 0)

In [ ]:
# Data for k-means ready 
# t_old.shape 

In [21]:
# save("/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/others/k_mean_data.pkl", t_old)
t_old = load("/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/others/k_mean_data.pkl")

In [22]:
# t_old.shape

(242550, 250)

# Applying K-mean 

In [ ]:
# kmeans = KMeans(n_clusters=2000, random_state=0).fit(t_old)

In [ ]:
# save("/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/others/k_mean_centers_2000.pkl", kmeans.cluster_centers_)
# save("/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/others/k_mean_onject_2000.pkl", kmeans)

# Generating Histograms and assignment maps 

In [16]:
# Loading image features, image names , kmean centers for 1000
k_mean_centers = load('/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/others/k_mean_centers_250_1000.pkl')


In [17]:
def input_vector_encoder(feature, codebook):
    """
    Input all the local feature of the image
    Pooling (encoding) by codebook and return
    """
    code, _ = vq.vq(feature, codebook)
    word_hist, bin_edges = np.histogram(code, bins=range(codebook.shape[0] + 1))
    return word_hist

In [18]:

def get_assignment_map(feature, codebook):
    """
    Input all the local feature of the image
    Pooling (encoding) by codebook and return
    """
    code, _ = vq.vq(feature, codebook)
    return code

In [ ]:
Hist_all = []

for idx,t in enumerate(train_f):
  t_new = np.reshape(t,(250,7*7)).T
  Hist_all.append(input_vector_encoder(t_new, k_mean_centers))
  

In [24]:
amap_all = []

for idx,t in enumerate(train_f):
  t_new = np.reshape(t,(250,7*7)).T 
  x = np.reshape(get_assignment_map(t_new, k_mean_centers),(7,7))
  amap_all.append(x)
  

In [ ]:
# save('/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/others/Hist_all',Hist_all )
# np.array(Hist_all).shape
# save('/content/drive/MyDrive/SUB/PROJECTS/IR/ParisDataV2/CNN_250/others/assignment_map_250_1000.pkl',amap_all )

Hist_all = load('/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/others/Hist_all')

# All Images 64,64 dict with image : resized pixels 

In [ ]:
# !ls /content/drive/MyDrive/SUB/PROJECTS/IR/data/

In [ ]:
# import cv2
# datapath = "/content/drive/MyDrive/SUB/PROJECTS/IR/data/"
# format = [".jpg",".png",".jpeg"]

# img_dict = {}

# num1 = 64
# num2 = 64
# count = 0
# path = os.path.join(datapath)

# for img in os.listdir(path):
#   if img.endswith(tuple(format)):
#     # print(img)
#     # train_name.append(img)
#     img2 = cv2.imread(os.path.join(path,img))
#     new_img = cv2.resize(img2, (num2, num1))
#     # train_img.append(new_img)
#     # print(new_img)
#     # print(img)
#     img_dict[img] = list(new_img)
#     count = count + 1
#     print(count)
#     # break

In [ ]:
# save("/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/others/data.pkl", img_dict)
img_dict = load("/content/drive/MyDrive/SUB/PROJECTS/IR/CNN_250/others/data.pkl")